## Gradient Boosting (梯度提升算法）

Boosting 提升算法的实质是在函数空间(损失函数)上做梯度下降。

"GB is performing gradient descent in (loss) function space."

GB 梯度提升算法可以用于分类和回归，它通过对一组弱分类器的集成。而弱分类器通常使用决策树（Decision Tree), 所以就有了我们的 GBDT (Gradient Boosting Decision Tree), 梯度提升决策树。


### 1. 算法

- Input: 
    - 给定样本集： $\{(x_i, y_i)\}_{i = 1}^{n}$
    - 可微损失函数：$L(y, F(x))$
    - 基学习器: $h_m(x)$
    - 迭代次数: $M$
- Algorithm:
    1. 使用一个常数来初始化模型
        $$
        F_0(x) = \underset{\gamma}{\arg\min} \sum_{i=1}^n L(y_i, \gamma).
        $$
    2. For m = to M:
        1. 根据损失函数计算残差
            $$
            r_{im} = -\left[\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}\right]_{F(x)=F_{m-1}(x)} \quad \mbox{for } i=1,\ldots,n.
            $$
        2. 根据残差组成新的样本集 $\{(x_i, r_{im})\}_{i=1}^n.$ 来训拟合出一个弱分类器 $h_m(x)$ 。
        3. 根据残差训练出了弱分类器，那么该弱分类器如何跟前 (m-1) 个弱分类器进行线性组合那？我们需要找出一个  $\gamma_m$，使得线性组合后的误差最小
        $$
        \gamma_m = \underset{\gamma}{\operatorname{arg\,min}} \sum_{i=1}^n L\left(y_i, F_{m-1}(x_i) + \gamma h_m(x_i)\right).
        $$
        4. 找到后，就可用新的线性组来来更新模型了
        $$
        F_{m}(x)=F_{{m-1}}(x)+\gamma _{m}h_{m}(x).    
        $$
    3. 输出 $F_{M}(x)$

### 2. 算法实现

看上面的算法也话感觉会很复杂，我们可以用 MSE 损失函数来举例说明一下。

即 $ L(y, F(x)) = \frac{1}{2}(y_i - F(x))^2 $, 为了方便，我们用 $ \hat{y} $ 来代替 $ F(x) $。

所有下面的实现算法。

- Input: 
    - 给定样本集： $ \{(x_i, y_i)\}_{i = 1}^{n} $
    - 可微损失函数：$ L(y, F(x)) = \frac{1}{2}(y_i - \hat{y_i})^2 $
    - 基学习器: $ h_m(x) $
    - 迭代次数: $ M $
- Algorithm:
    1. 使用一个常数来初始化模型
        $$
        F_0(x) = \underset{\gamma}{\arg\min} \sum_{i=1}^n L(y_i, \gamma).
        $$
        
        寻找这个常数 $\gamma$，一个直观的想法用平均值就好，即 $\gamma = \frac{1}{n}\sum_{i = 1}^{n}y_i$ 。
        
        其实这是可以推导的，因为 $F_0(x) = \sum_{i=1}^n L(y_i, \gamma) = \frac{1}{2} \sum_{i = 1}(y_i - r)^2$。对该函数示驻点即可：
        $$
            \frac{dF_0}{d \gamma} = -\sum_{i = 1}^{n}(y_i - \gamma)
        $$
        
        所以，$ \gamma = \frac{1}{n}\sum_{i = 1}^{n}y_i $
        
    2. For m = to M:
        1. 根据损失函数计算残差
            $$
            r_{im} = -\left[\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}\right]_{F(x)=F_{m-1}(x)} \quad \mbox{for } i=1,\ldots,n.
            $$
            
            根据 $L = \frac{1}{2}(y_i - \hat{y_i})^2$ 为 MSE 函数，可以求得：
            
            $$
                \frac{\partial L}{\partial y_i} = -(y_i - \hat{y_i})
            $$
            
            所以 $r_{im} =(y_i - \hat{y_i}), i = 1, \dots, n $   
        2. 根据残差组成新的样本集 $\{(x_i, r_{im})\}_{i=1}^n.$ 来训拟合出一个弱分类器 $h_m(x)$ 。
        3. 计算出参数 $\gamma_m$
        $$
        \gamma_m = \underset{\gamma}{\operatorname{arg\,min}} \sum_{i=1}^n L\left(y_i, F_{m-1}(x_i) + \gamma h_m(x_i)\right).
        $$
        最终可以求得:
        $$
        \gamma_m = mean(y_i - F_{m - 1}(x_i))
        $$
        4. 更新模型
        $$
        F_{m}(x)=F_{{m-1}}(x)+\gamma _{m}h_{m}(x).    
        $$
    3. 输出 $F_{M}(x)$



有了 $r$, $r_m$ 的求法，我们就可实现我们的算法了。

对最不同的损失函数时，如: MSE, MSE，  $r$, $r_m$ 的更新公式需要自己再算一遍才行。
